In [ ]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from sklearn.metrics import accuracy_score, classification_report
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [ ]:
# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


In [ ]:
class CatDogDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        
        # 고양이 이미지 (label: 0)
        cat_dir = os.path.join(root_dir, 'cats')
        if os.path.exists(cat_dir):
            for img_name in os.listdir(cat_dir):
                if img_name.endswith(('.jpg', '.jpeg', '.png')):
                    self.images.append(os.path.join(cat_dir, img_name))
                    self.labels.append(0)
        
        # 개 이미지 (label: 1)
        dog_dir = os.path.join(root_dir, 'dogs')
        if os.path.exists(dog_dir):
            for img_name in os.listdir(dog_dir):
                if img_name.endswith(('.jpg', '.jpeg', '.png')):
                    self.images.append(os.path.join(dog_dir, img_name))
                    self.labels.append(1)
        
        print(f"Loaded {len(self.images)} images")
        print(f"Cats: {self.labels.count(0)}, Dogs: {self.labels.count(1)}")
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        
        # 이미지 로드 및 전처리
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [ ]:
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes):
        super(FCNN, self).__init__()
        
        layers = []
        prev_size = input_size
        
        # 은닉층들
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.3))  # 과적합 방지
            prev_size = hidden_size
        
        # 출력층
        layers.append(nn.Linear(prev_size, num_classes))
        
        self.network = nn.Sequential(*layers)
        
        # 가중치 초기화
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
    
    def forward(self, x):
        # 입력을 1차원으로 평탄화 (FCNN의 핵심 특징)
        x = x.view(x.size(0), -1)
        return self.network(x)


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    
    for epoch in range(num_epochs):
        # 학습 모드
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            train_total += target.size(0)
            train_correct += (predicted == target).sum().item()
            
            if batch_idx % 50 == 0:
                print(f'Epoch: {epoch+1}/{num_epochs}, Batch: {batch_idx}/{len(train_loader)}, '
                      f'Loss: {loss.item():.4f}')
        
        # 검증 모드
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                
                val_loss += loss.item()
                _, predicted = torch.max(output.data, 1)
                val_total += target.size(0)
                val_correct += (predicted == target).sum().item()
        
        # 평균 손실과 정확도 계산
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        train_accuracy = 100 * train_correct / train_total
        val_accuracy = 100 * val_correct / val_total
        
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%')
        print(f'  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')
        print('-' * 50)
    
    return train_losses, val_losses, train_accuracies, val_accuracies

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    accuracy = accuracy_score(all_targets, all_predictions)
    report = classification_report(all_targets, all_predictions, 
                                 target_names=['Cat', 'Dog'])
    
    return accuracy, report

def plot_training_history(train_losses, val_losses, train_accuracies, val_accuracies):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # 손실 그래프
    ax1.plot(train_losses, label='Train Loss')
    ax1.plot(val_losses, label='Validation Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True)
    
    # 정확도 그래프
    ax2.plot(train_accuracies, label='Train Accuracy')
    ax2.plot(val_accuracies, label='Validation Accuracy')
    ax2.set_title('Training and Validation Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.savefig('fcnn_training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
train_dir = 'data/archive/training_set/training_set'
test_dir = 'data/archive/test_set/test_set'

In [ ]:
img_size = 64  # 64x64로 설정하여 계산량 줄이기
    
    # 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])

# 데이터셋 로드
print("Loading datasets...")
train_dataset = CatDogDataset(train_dir, transform=transform)
test_dataset = CatDogDataset(test_dir, transform=transform)

# 데이터 로더 생성
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                        shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, 
                       shuffle=False, num_workers=2)

# FCNN 모델 생성
input_size = img_size * img_size * 3  # RGB 3채널
hidden_sizes = [512, 256, 128]  # 은닉층 크기
num_classes = 2  # 고양이, 개

print(f"Input size: {input_size}")
print(f"Hidden sizes: {hidden_sizes}")

In [ ]:
model = FCNN(input_size, hidden_sizes, num_classes).to(device)
    
# 모델 정보 출력
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)



In [ ]:
# 학습 시작
print("\nStarting FCNN training...")
start_time = time.time()

train_losses, val_losses, train_accuracies, val_accuracies = train_model(
    model, train_loader, test_loader, criterion, optimizer, num_epochs=10
)

training_time = time.time() - start_time
print(f"\nTraining completed in {training_time:.2f} seconds")

# 모델 평가
print("\nEvaluating model...")
test_accuracy, test_report = evaluate_model(model, test_loader)

print(f"\nTest Accuracy: {test_accuracy:.4f}")
print("\nClassification Report:")
print(test_report)

# 학습 과정 시각화
plot_training_history(train_losses, val_losses, train_accuracies, val_accuracies)

# 모델 저장
torch.save(model.state_dict(), 'fcnn_cat_dog_model.pth')
print("\nModel saved as 'fcnn_cat_dog_model.pth'")

# FCNN의 특징 요약
print("\n" + "="*60)
print("FCNN 모델 특징 요약:")
print("="*60)
print(f"1. 입력 크기: {img_size}x{img_size}x3 = {input_size} (평탄화됨)")
print(f"2. 총 매개변수: {total_params:,}")
print(f"3. 은닉층 구조: {hidden_sizes}")
print(f"4. 최종 테스트 정확도: {test_accuracy:.4f}")
print(f"5. 학습 시간: {training_time:.2f}초")
print("\nFCNN의 한계점:")
print("- 공간적 정보 손실 (이미지가 1차원으로 평탄화됨)")
print("- 많은 매개변수로 인한 과적합 위험")
print("- 위치 변화에 민감함")
print("="*60)